<a href="https://colab.research.google.com/github/EnmaAii/Ex.1_Digital_Culture/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22Ex_1_Digital_Culture%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



1.   **Загрузка датасета и присвоение номера категории**




In [ ]:
from google.colab import files
files.upload()

In [ ]:
import pandas as pd
data = pd.read_csv("ag_news.csv", sep=';')
data.head()

,category,content
0,IT,webify helps insurance carriers stretch legac...
1,business,"update 1 thursday #39 s commodities roundup ,..."
2,business,california regulator is suing four insurers a...
3,crime,mexico fta paves way for asian trade agreemen...
4,sport,"t amp t , costa rica through , angus eve scor..."




2.   **Обработка данных**






In [ ]:
data.columns[data.isna().any()].tolist() #проверить наличие NaN в DataFrame, если массив пуст, то и пропущенных данных нет

[]

In [ ]:
data["category"] = data["category"].astype('category') # вместо хранения каждого уникального текстового значения, pandas будет хранить уникальные категории и присваивать каждой категории числовой код.
data["category"] = data["category"].cat.codes # извлекаем числовые коды, присвоенные каждой переменной
data.head()

,category,content
0,0,webify helps insurance carriers stretch legac...
1,1,"update 1 thursday #39 s commodities roundup ,..."
2,1,california regulator is suing four insurers a...
3,2,mexico fta paves way for asian trade agreemen...
4,3,"t amp t , costa rica through , angus eve scor..."


In [ ]:
category_count = data["category"].max() + 1
category_count

4

In [ ]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer('basic_english')
# get_tokenizer - функция для обработки текста
# аргумент "basic_english" - это формат обработки (приведение к нижнему регистру, избавление от спецсимволов, ...)
# этот же аргумент можно указывать при обработке текстов на других языках!

def yield_tokens(data_train):
  for text in data_train:
    yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(data["content"]), specials=["<unk>"]) # здесь data_train - тренировочные данные
vocab.set_default_index(vocab["<unk>"])  # какой индекс нужно вернуть, если слово не найдено в словаре

In [ ]:
vocab(['feel', 'i', 'say', 'hello', 'hi']) # проверка

[2842, 282, 233, 12544, 24104]



3.   **Создание классов для загрузки данных и разделение данных на тренировочные / тестировочные**




In [ ]:
from torch.utils.data import Dataset # для работы с датасетом
import torch # для НС и тд

class TextDataset(Dataset):
    def __init__(self, data_text, y): # конструктор класса
        self.data = data_text # сам текст
        self.y = y # язык

    def __len__(self):   # len(trainset)
        return len(self.data)

    def __getitem__(self, idx): # data[0]
      text = vocab(tokenizer(self.data.iloc[idx]))
      return torch.tensor(text), torch.tensor(self.y.iloc[idx], dtype=torch.int64)

In [ ]:
y = data["category"]
data = data['content']

In [ ]:
from sklearn.model_selection import train_test_split
data_train, data_test, \
  y_train, y_test = train_test_split(data, y, test_size=0.3)

In [ ]:
trainset = TextDataset(data_text=data_train, y=y_train)
testset = TextDataset(data_text=data_test, y=y_test)

In [ ]:
def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_text, _label) in batch:
         label_list.append(_label)
         processed_text = torch.tensor(_text, dtype=torch.int64)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list, text_list, offsets

In [ ]:
# Загрузчики в НС
from torch.utils.data import DataLoader
trainloader = DataLoader(trainset, batch_size=8, shuffle=False,
                         collate_fn=collate_batch)

testloader = DataLoader(testset, batch_size=8, shuffle=False,
                        collate_fn=collate_batch)


In [ ]:
num_class = len(set(y_train))
vocab_size = len(vocab)
print("Кол-во классов: ", num_class)
print("Размер словаря: ", vocab_size)

Кол-во классов:  4
Размер словаря:  95811





4.    **Создание НС**



In [ ]:
from torch import nn

class TextClassificationModel(nn.Module):
    def __init__(self):
        super(TextClassificationModel, self).__init__() # вытаскивает методы из nn.Module
        self.bag = nn.EmbeddingBag(vocab_size, 64, sparse=True)  # слой-матрица
        # EmbeddingBag - для восприятия слов в виде двоичных векторов
        self.lin = nn.Linear(64, 4)  # на вход подаётся 64 числа - выход от self.bag, слой выдаёт 4 числа - вероятность каждой новости, их 4 типа
    def forward(self, text, offsets): # проход через НС
        return self.lin( self.bag(text, offsets) )

net = TextClassificationModel()

import torch.optim as optim
# В качестве loss function используем кросс-энтропию
criterion = nn.CrossEntropyLoss()

# В качестве оптимизатора - стохастический градиентный спуск
optimizer = optim.SGD(net.parameters(), lr=0.1)




5.   **Обучение НС**



In [ ]:
losses = []
running_corrects = 0
net.train(True)
for epoch in range(70):
    running_loss = 0.0
    running_corrects = 0.0
    for i, data in enumerate(trainloader, 0):
        labels, inputs, offsets = data

        optimizer.zero_grad() # включаю оптимизатор
        outputs = net(inputs, offsets) # проход данных через НС
        _, preds = torch.max(outputs.data, 1) # поиск ответа
        loss = criterion(outputs, labels) # считаю log loss
        loss.backward()
        optimizer.step() #меняю веса

        running_loss += loss.item()
        # считаю accuracy:
        running_corrects += int(torch.sum(preds == labels.data)) / len(labels)
        if i % 10000 == 9999:
          print('[%d, %5d] loss: %.3f accuracy: %.3f' % (epoch + 1, i + 1, running_loss/10000, running_corrects/10000 ))
          losses += [running_loss/10000]
          running_loss = 0.0
          running_corrects = 0.0

print('Finished Training')

<ipython-input-23-c54194b7cad6>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  processed_text = torch.tensor(_text, dtype=torch.int64)


[1, 10000] loss: 1.041 accuracy: 0.569
[2, 10000] loss: 0.742 accuracy: 0.717
[3, 10000] loss: 0.613 accuracy: 0.774
[4, 10000] loss: 0.541 accuracy: 0.805
[5, 10000] loss: 0.495 accuracy: 0.823
[6, 10000] loss: 0.462 accuracy: 0.837
[7, 10000] loss: 0.437 accuracy: 0.847
[8, 10000] loss: 0.417 accuracy: 0.855
[9, 10000] loss: 0.401 accuracy: 0.861
[10, 10000] loss: 0.387 accuracy: 0.867
[11, 10000] loss: 0.375 accuracy: 0.871
[12, 10000] loss: 0.364 accuracy: 0.875
[13, 10000] loss: 0.354 accuracy: 0.879
[14, 10000] loss: 0.346 accuracy: 0.882
[15, 10000] loss: 0.338 accuracy: 0.885
[16, 10000] loss: 0.331 accuracy: 0.887
[17, 10000] loss: 0.324 accuracy: 0.890
[18, 10000] loss: 0.318 accuracy: 0.892
[19, 10000] loss: 0.313 accuracy: 0.894
[20, 10000] loss: 0.307 accuracy: 0.896
[21, 10000] loss: 0.302 accuracy: 0.898
[22, 10000] loss: 0.298 accuracy: 0.900
[23, 10000] loss: 0.293 accuracy: 0.902
[24, 10000] loss: 0.289 accuracy: 0.903
[25, 10000] loss: 0.284 accuracy: 0.904
[26, 1000

In [ ]:
outputs

tensor([[-3.6765, -3.1626,  1.5211,  5.3695],
        [ 2.0471,  7.5901, -0.4475, -9.1890],
        [ 8.4393,  0.9437, -0.2242, -9.1695],
        [ 9.9480, -1.2216, -0.5728, -8.0629],
        [ 0.0120, -3.6118, -1.7225,  5.1885],
        [ 1.1676, -1.2469,  2.9649, -2.9286],
        [ 6.4421, -0.9761, -0.1365, -5.4864],
        [-5.6454, -5.4070,  2.7227,  8.3574]], grad_fn=<AddmmBackward0>)

In [ ]:
preds

tensor([3, 1, 0, 0, 3, 2, 0, 3])



6. **Тестирование**



In [ ]:
net.train(False)
runninig_correct = 0
num_of_tests = 0
for data in testloader:
    labels, inputs, offsets = data

    output = net(inputs, offsets)
    _, predicted = torch.max(output, 1)

    runninig_correct += int(torch.sum(predicted == labels)) / len(labels)
    num_of_tests += 1

print(runninig_correct / num_of_tests)

<ipython-input-23-c54194b7cad6>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  processed_text = torch.tensor(_text, dtype=torch.int64)


0.8874166666666666
